In [5]:
# construct a Prefix Trie and Suffix Trie
# store (appending) weight value on each path (similar to HashMap + LinkedList)
# Runtime: 968 ms
# https://leetcode.com/problems/prefix-and-suffix-search/discuss/320712/Different-Python-solutions-with-thinking-process
from collections import defaultdict
from typing import List

class TrieNode:
    def __init__(self):
        self.children = defaultdict(TrieNode)
        self.weights = []
        # no need to have isLeaf flag here because we only care about the prefix path rather than where to end

class Trie:
    def __init__(self):
        self.root = TrieNode()
    
    def insert(self, word: str, i: int) -> None:
        node = self.root
        node.weights.append(i)
        for ch in word:
            node = node.children[ch]
            node.weights.append(i)
    
    def search(self, prefix: str) -> bool:
        node = self.root
        for ch in prefix:
            node = node.children.get(ch)
            if not node:
                return []
        return node.weights
    
class WordFilter:
    def __init__(self, words: List[str]):
        self.preTrie, self.postTrie = Trie(), Trie()
        for i, w in enumerate(words):
            self.preTrie.insert(w, i)
            self.postTrie.insert(w[::-1], i)

    def f(self, prefix: str, suffix: str) -> int:
        pres, posts = self.preTrie.search(prefix), self.postTrie.search(suffix[::-1])
        # Two pointers, place each pointer on the tail of each weights array (in ascending order) 
        # find the common maximum weight by comparing pres[i] and posts[j], 
        # if not equal, move the pointer on the larger weight forward
        i, j = len(pres)-1, len(posts)-1
        while i >= 0 and j >= 0:
            if pres[i] == posts[j]:
                return pres[i]
            if pres[i] > posts[j]:
                i -= 1
            else:
                j -= 1
        return -1

In [16]:
# construct a Prefix Trie and Suffix Trie
# store the max weight value on leaf node;
# find the prefix node, and then dfs to get weight of all words under it;
# intersect two weight sets and find the maximum weight
# Runtime: 2336 ms
from collections import defaultdict
from typing import List, Set

class TrieNode:
    def __init__(self):
        self.children = defaultdict(TrieNode)
        self.weight = -1 # -1 means isLeaf == False

class Trie:
    def __init__(self):
        self.root = TrieNode()
    
    def insert(self, word: str, i: int) -> None:
        node = self.root
        for ch in word:
            node = node.children[ch]
        node.weight = i
    
    def findAllWeights(self, prefix: str) -> Set[int]:
        node = self.root
        wgts = set()
        # locate the prefix node
        for ch in prefix:
            node = node.children.get(ch)
            if not node:
                return wgts
        # dfs() to find all leaf nodes whose ancestor is the prefix node above
        def dfs(node: TrieNode):
            if node.weight != -1:
                wgts.add(node.weight)
            for child in node.children.values():
                dfs(child)
                
        dfs(node)
        return wgts
    
class WordFilter:
    def __init__(self, words: List[str]):
        self.preTrie, self.postTrie = Trie(), Trie()
        for i, w in enumerate(words):
            self.preTrie.insert(w, i)
            self.postTrie.insert(w[::-1], i)

    def f(self, prefix: str, suffix: str) -> int:
        pres, posts = self.preTrie.findAllWeights(prefix), self.postTrie.findAllWeights(suffix[::-1])
#         print(pres, posts)
        if pres and posts:
            return max(pres & posts)
        if pres and not suffix: # suffix is ''
            return max(pres)
        if posts and not prefix: # prefix is ''
            return max(posts)
        return -1

In [17]:
# Your WordFilter object will be instantiated and called as such:
words = ['apple', 'aple', 'apbble']
obj = WordFilter(words)
print(obj.f(prefix='ap',suffix='le'))

2
